In [1]:
from sentence_transformers import SentenceTransformer
import pinecone
import numpy as np

In [2]:
pinecone.init(api_key="91e41499-ee04-4431-8ddc-d878f12a62a0", environment="us-west4-gcp-free")

In [8]:
pinecone.create_index("chatbot", dimension=384, metric="cosine")

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=UTF-8', 'date': 'Wed, 05 Jul 2023 09:55:06 GMT', 'x-envoy-upstream-service-time': '1281', 'content-length': '131', 'server': 'envoy'})
HTTP response body: The index exceeds the project quota of 1 pods by 1 pods. Upgrade your account or change the project settings to increase the quota.


In [3]:
pinecone.list_indexes()
index = pinecone.Index("chatbot")

In [6]:
sentences =  [
    "What is Foliogrow",
    "Who are the software partners",
    "What is the contact address",
    "Who invented Foliogrow",
    "Who created Foliogrow"
    
    ]

# answers = [
#     "FolioGrow is smart cannabis cultivation software that helps you make the right decisions to increase your yields and profits",
#     "metrc,ridder,growlink,netexam,salesforce,IBMwatson",
#     " 3002 Commerce Street Dallas, TX 75226"
# ]

# urls = [
#     "https://foliogrow.com/",
#     "https://foliogrow.com/metrc/",
#     "https://foliogrow.com/contact/"
# ]

In [7]:
model = SentenceTransformer('./all-MiniLM-L6-v2')
embeddings = []
for x in sentences:
    embedding = model.encode(x)
    embeddings.append(embedding)
embeddings_array = np.array(embeddings)
print(embeddings_array)
print(len(embeddings_array[0]))
    





[[-0.01210796  0.03986302 -0.11864929 ... -0.04804447  0.09362378
   0.03605172]
 [-0.05488758 -0.03095406 -0.01984533 ... -0.07090454  0.09768245
   0.01130756]
 [-0.13744348 -0.00276467  0.03286193 ...  0.03753722 -0.04565042
   0.02908565]
 [-0.04412768  0.067283   -0.10037435 ... -0.03088048  0.07547478
   0.02491976]
 [-0.05599381  0.03485883 -0.09797404 ... -0.01528921  0.06636149
   0.03234956]]
384


In [8]:
embeddings_array =  embeddings_array.tolist()


In [10]:
index.upsert([
    ("A", embeddings_array[0], {"answer": "FolioGrow is smart cannabis cultivation software that helps you make the right decisions to increase your yields and profits", "url": "https://foliogrow.com/"}),
    ("B", embeddings_array[1], {"answer": "metrc,ridder,growlink,netexam,salesforce,IBMwatson", "url": "https://foliogrow.com/metrc/"}),
    ("C", embeddings_array[2], {"answer": "3002 Commerce Street Dallas, TX 75226", "url": "https://foliogrow.com/metrc/"}),
    ("D", embeddings_array[3], {"answer": "MediaDefined", "url": "https://www.mediadefined.lk/"})
      
])

{'upserted_count': 4}

In [10]:
index.query(
  vector=embeddings_array[4],
  top_k=1,
  include_values=False,
  include_metadata=True
)

{'matches': [{'id': 'D',
              'metadata': {'answer': 'MediaDefined',
                           'url': 'https://www.mediadefined.lk/'},
              'score': 0.931967437,
              'values': []}],
 'namespace': ''}